### Import libs

In [74]:
from types import MethodDescriptorType
import requests
import hashlib
import threading
import json
import pandas as pd


import matplotlib.pyplot as plt
    
import datetime as dt  

### Set parameters

In [75]:
root_url = 'https://api.binance.com/api/v1/klines'

symbol = 'ETHUSDT'

interval = '1d'

window = 10

start_date = '2020-01-01'
#end_date = datetime.now().strftime('%y-%d-%m')

url = root_url + '?symbol=' + symbol + '&interval=' + interval
print(url)

https://api.binance.com/api/v1/klines?symbol=ETHUSDT&interval=1d


### Data in

In [76]:
def get_bars(symbol, interval):
    url = root_url + '?symbol=' + symbol + '&interval=' + interval
    data = dict()
    data['limit'] = 1000
    data = json.loads(requests.get(url,params=data).text)
    df = pd.DataFrame(data)
    df.columns = ['open_time',
                 'open', 'high', 'low', 'close', 'volume',
                 'close_time', 'qav', 'num_trades',
                 'taker_base_vol', 'taker_quote_vol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.close_time]
    return df

In [141]:
df = get_bars(symbol,interval)

### Rolling Mean-Std 

In [142]:
# change data types  
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric)


columns = ['open','high','low','close', 'volume']
df = df[columns]

# measure rolling mean/std
df['MA'] = df['high'].rolling(window).mean()
df['STD'] = df['high'].rolling(window).std()


In [143]:
df.head(2)

,open,high,low,close,volume,MA,STD
2018-12-26 01:59:59.999,138.01,139.26,119.68,127.60,1.076164e+06,NaN,NaN
2018-12-27 01:59:59.999,127.61,135.32,122.51,129.35,7.948341e+05,NaN,NaN


### Price changes

In [144]:
# Seeing how much price changed in consecutive days (in percentage)
#df_index = df.index
df = df.reset_index()
df['Price_Change']=0
for i in range(len(df)-1):
    df['Price_Change'][i]=((df['close'][i+1] - df['close'][i])/df['close'][i])*100
    


<ipython-input-144-9b638e29dfda>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [164]:
df.rename(columns={'index': 'Date'}, inplace=True)
df['Date'] = df['Date'].dt.date
df.Date = pd.to_datetime(df.Date)
df.head()

,Date,open,high,low,close,volume,MA,STD,Price_Change,Change,high_plus_2std,high_minus_2std,above_boundary,below_boundary
0,2018-12-26,138.01,139.26,119.68,127.60,1.076164e+06,NaN,NaN,1,Neutral,NaN,NaN,False,False
1,2018-12-27,127.61,135.32,122.51,129.35,7.948341e+05,NaN,NaN,-12,Decrease,NaN,NaN,False,False
2,2018-12-28,129.35,131.09,111.22,113.80,7.521779e+05,NaN,NaN,18,Increase,NaN,NaN,False,False
3,2018-12-29,113.75,136.66,111.87,135.30,8.944542e+05,NaN,NaN,-1,Neutral,NaN,NaN,False,False
4,2018-12-30,135.30,145.42,128.50,132.89,1.006654e+06,NaN,NaN,3,Neutral,NaN,NaN,False,False


In [146]:
# Change in words
df['Change']=0
for i in range(len(df['Price_Change']-1)):
    if (df['Price_Change'][i] >= 5):
        df['Change'][i] = "Increase"
    elif (df['Price_Change'][i] < 5) and (df['Price_Change'][i] > -5):
        df['Change'][i] = "Neutral"
    else:
        df['Change'][i] = "Decrease"

<ipython-input-146-599b46811889>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/user/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-146-599b46811889>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-146-599b46811889>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the

In [147]:
df.head()

,Date,open,high,low,close,volume,MA,STD,Price_Change,Change
0,2018-12-26 01:59:59.999,138.01,139.26,119.68,127.60,1.076164e+06,NaN,NaN,1,Neutral
1,2018-12-27 01:59:59.999,127.61,135.32,122.51,129.35,7.948341e+05,NaN,NaN,-12,Decrease
2,2018-12-28 01:59:59.999,129.35,131.09,111.22,113.80,7.521779e+05,NaN,NaN,18,Increase
3,2018-12-29 01:59:59.999,113.75,136.66,111.87,135.30,8.944542e+05,NaN,NaN,-1,Neutral
4,2018-12-30 01:59:59.999,135.30,145.42,128.50,132.89,1.006654e+06,NaN,NaN,3,Neutral


### Dates where price exceeded 2 stds 

In [148]:
df['high_plus_2std'] = df.MA + 2 * df.STD
df['high_minus_2std'] = df.MA - 2 * df.STD

In [149]:
import numpy as np
df['above_boundary'] = np.where(df['high_plus_2std'] < df['high'] , True, False)
df['below_boundary'] = np.where(df['high_minus_2std'] > df['high'] , True, False)

In [150]:
df.head(2)

,Date,open,high,low,close,volume,MA,STD,Price_Change,Change,high_plus_2std,high_minus_2std,above_boundary,below_boundary
0,2018-12-26 01:59:59.999,138.01,139.26,119.68,127.60,1.076164e+06,NaN,NaN,1,Neutral,NaN,NaN,False,False
1,2018-12-27 01:59:59.999,127.61,135.32,122.51,129.35,7.948341e+05,NaN,NaN,-12,Decrease,NaN,NaN,False,False


In [151]:
tweet_vol = pd.read_csv('../data/total_tweets.csv')
tweet_vol.Date = pd.to_datetime(tweet_vol.Date)

tweet_vol.head()

,Date,Total_Tweets
0,2016-03-16,1185.0
1,2016-03-17,1211.0
2,2016-03-18,1123.0
3,2016-03-19,943.0
4,2016-03-20,1304.0


In [168]:
# merge tweets on main dataset
final = pd.merge(left=df, right=tweet_vol, how='left', on='Date')

In [173]:
final.head()

,Date,open,high,low,close,volume,MA,STD,Price_Change,Change,high_plus_2std,high_minus_2std,above_boundary,below_boundary,Total_Tweets
0,2018-12-26,138.01,139.26,119.68,127.60,1.076164e+06,NaN,NaN,1,Neutral,NaN,NaN,False,False,5472.0
1,2018-12-27,127.61,135.32,122.51,129.35,7.948341e+05,NaN,NaN,-12,Decrease,NaN,NaN,False,False,6689.0
2,2018-12-28,129.35,131.09,111.22,113.80,7.521779e+05,NaN,NaN,18,Increase,NaN,NaN,False,False,5983.0
3,2018-12-29,113.75,136.66,111.87,135.30,8.944542e+05,NaN,NaN,-1,Neutral,NaN,NaN,False,False,6636.0
4,2018-12-30,135.30,145.42,128.50,132.89,1.006654e+06,NaN,NaN,3,Neutral,NaN,NaN,False,False,5866.0


In [180]:
# tweet vs change price
import plotly.express as px
fig = px.scatter(final, x="Date", y="Price_Change", color="above_boundary")
fig.show()


In [181]:
fig = px.scatter(final, x="Total_Tweets", y="Price_Change", color="above_boundary")
fig.show()


### Plot candlesticks

In [47]:
import plotly.graph_objects as go

from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.update_layout(
    yaxis_title="USDT",
    title={
        'text': symbol + " Price",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df.MA,
        name = "Moving average",
        mode="lines",
        line=go.scatter.Line(color="yellow"),
        showlegend=True)
)


fig.add_trace(
    go.Scatter(
        x=df.index,
        y= df['MA'] + 2 * df.STD,
        name = "Upper bound",
        mode="lines",
        line=go.scatter.Line(color="goldenrod"),
        showlegend=True)
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y= df['MA'] - 2 * df.STD,
        name = "Lower bound",
        mode="lines",
        line=go.scatter.Line(color="magenta"),
        showlegend=True)
)

fig.add_trace(go.Scatter(
    x= df[df['above_boundary'] == True].index,
    y= df.close,
    marker=dict(color="green", size=8),
    mode="markers",
    name = 'above_boundary'
))


fig.add_trace(go.Scatter(
    x= df[df['below_boundary'] == True].index,
    y= df.close,
    marker=dict(color="crimson", size=8),
    mode="markers",
    name = 'below_boundary'
))



fig.show()

### Save plot local

In [172]:
# # save the plot in the path you want! 
# fig.write_html("eth_usdt.html")